# **Personalized Movie Recommendations**
The best online marketplaces create a seamless shopping experience by predicting what customers want based on their recent behavior.

In [ ]:
!pip3 install thirdai
!pip3 install pandas # Needed by the datasets module to convert datasets to CSV format
!pip3 install numpy

In [ ]:
import utils

train_filename, test_filename, inference_batch, index_batch = utils.download_movielens()

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "userId": bolt.types.categorical(n_unique_classes=6040),
        "movieTitle": bolt.types.categorical(n_unique_classes=3706),
        "timestamp": bolt.types.date(),
    },
    temporal_tracking_relationships={
        "userId": ["movieTitle"]
    },
    target="movieTitle",
)

In [ ]:
train_config = (bolt.TrainConfig(epochs=3, learning_rate=0.001)
                    .with_metrics(["recall@10"]))

model.train(train_filename, train_config)

eval_config = (bolt.EvalConfig()
                   .with_metrics(["recall@1", "recall@10", "recall@100"]))

model.evaluate(test_filename, eval_config);

In [ ]:
print(inference_batch)

In [ ]:
import numpy as np

prediction = model.predict(inference_batch[0])
class_name = model.class_name(np.argmax(prediction))
print("Single Prediction Result")
print("Input:", inference_batch[0], "Prediction:", class_name, "\n")

prediction_batch = model.predict_batch(inference_batch)
class_ids = np.argmax(prediction_batch, axis=1)
class_names = [model.class_name(class_id) for class_id in class_ids]
print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)

In [ ]:
print(index_batch)

In [ ]:
model.reset_temporal_trackers()

for input_sample, ground_truth in zip(inference_batch, index_batch):
    prediction = model.predict(input_sample)
    top_10_ids = np.argsort(-prediction)[:10]

    top_10_class_names = [model.class_name(class_id) for class_id in top_10_ids]
    
    print("Single Prediction Result")
    print("Input:", input_sample, "Predictions:", top_10_class_names)
    
    model.index(ground_truth)
    print("Indexed ground truth sample:", ground_truth, "\n")

